# balanced network

Let's start with a rather complete example modified from 

http://neuralensemble.org/docs/PyNN/examples/VAbenchmarks.html


In [1]:
%run 2_boilerplate.ipynb

CSAConnector: libneurosim support not available in NEST.
Falling back on PyNN's default CSAConnector.
Please re-compile NEST using --with-libneurosim=PATH


/home/laurent/.local/lib/python3.8/site-packages/pyNN/nest/__init__.py:53: UserWarning:Unable to install NEST extensions. Certain models may not be available.


Python implementation: CPython
Python version       : 3.8.5
IPython version      : 7.18.1

numpy     : 1.19.2
matplotlib: 3.3.2
pyNN      : 0.9.5
neo       : 0.8.0

Compiler    : GCC 9.3.0
OS          : Linux
Release     : 5.4.0-53-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 4
Architecture: 64bit

Hostname: inv-ope-de06

Git hash: 6ae442d6e5aefed07811da70356fd7373c626aaf

Git repo: https://github.com/SpikeAI/2020-11_brainhack_Project7

Git branch: main



## initialization

In [2]:

from pyNN.utility import Timer, ProgressBar, init_logging, normalized_filename
from pyNN.random import NumpyRNG, RandomDistribution

init_logging(None, debug=False)

timer = Timer()


## running it...

In [3]:
node_id = sim.setup(timestep=dt, min_delay=delay, max_delay=1.0, **extra)
num_process = sim.num_processes()

timer.start()

print("%s Creating cell populations..." % node_id)

# create a single population of neurons, and then use population views to define
# excitatory and inhibitory sub-populations
celltype = sim.IF_cond_exp
all_cells = sim.Population(n_exc + n_inh, celltype(**cell_params), label="All Cells")
exc_cells = all_cells[:n_exc]
exc_cells.label = "Excitatory cells"
inh_cells = all_cells[n_exc:]
inh_cells.label = "Inhibitory cells"

ext_stim = sim.Population(20, sim.SpikeSourcePoisson(rate=rate, duration=stim_dur), label="expoisson")
rconn = 0.01
ext_conn = sim.FixedProbabilityConnector(rconn)
ext_syn = sim.StaticSynapse(weight=0.1)

print("%s Initialising membrane potential to random values..." % node_id)
rng = NumpyRNG(seed=rngseed, parallel_safe=parallel_safe)
uniformDistr = RandomDistribution('uniform', low=v_reset, high=v_thresh, rng=rng)
all_cells.initialize(v=uniformDistr)

print("%s Connecting populations..." % node_id)
progress_bar = ProgressBar(width=20)
connector = sim.FixedProbabilityConnector(pconn, rng=rng, callback=progress_bar)
exc_syn = sim.StaticSynapse(weight=w_exc, delay=delay)
inh_syn = sim.StaticSynapse(weight=w_inh, delay=delay)

connections = {}
connections['exc'] = sim.Projection(exc_cells, all_cells, connector, exc_syn, receptor_type='excitatory')
connections['inh'] = sim.Projection(inh_cells, all_cells, connector, inh_syn, receptor_type='inhibitory')
connections['ext'] = sim.Projection(ext_stim, all_cells, ext_conn, ext_syn, receptor_type='excitatory')

# === Setup recording ==========================================================
print("%s Setting up recording..." % node_id)
all_cells.record('spikes')
exc_cells[[0, 1]].record('v')

buildCPUTime = timer.diff()

# === Run simulation ===========================================================
print("%d Running simulation..." % node_id)
sim.run(tstop)

E_count = exc_cells.mean_spike_count()
I_count = inh_cells.mean_spike_count()

n_connections = "%d e→e,i  %d i→e,i" % (connections['exc'].size(),
                                      connections['inh'].size())
simCPUTime = timer.diff()
# === Output results to file ====================================================

print("%d Writing data to file..." % node_id)

root, basename, extension = "Results", "VAbenchmarks_exc", "pkl"
filename = normalized_filename(root, basename, extension, simulator, do_iso8601=True)

exc_cells.write_data(filename)
inh_cells.write_data(filename.replace("exc", "inh"))

writeCPUTime = timer.diff()

print("\n--- Vogels-Abbott Network Simulation ---")
print("Nodes                  : %d" % 1)
print("Number of Neurons      : %d" % n)
print("Number of Synapses     : %s" % n_connections)
print("Excitatory conductance : %g nS" % Gexc)
print("Inhibitory conductance : %g nS" % Ginh)
print("Excitatory rate        : %g Hz" % (E_count * 1000.0 / tstop,))
print("Inhibitory rate        : %g Hz" % (I_count * 1000.0 / tstop,))
print("Build time             : %g s" % buildCPUTime)
print("Simulation time        : %g s" % simCPUTime)
print("Writing time           : %g s" % writeCPUTime)


# === Finished with simulator ==================================================

sim.end()

/usr/lib/python3.8/dist-packages/nest/lib/hl_api_helper.py:127: UserWarning:
ResetNetwork is deprecated and will be removed in NEST 3.0.
2020-12-02 15:14:11,370 WARNING  [PyNN] NEST does not allow setting an initial value for g_ex (/home/laurent/.local/lib/python3.8/site-packages/pyNN/nest/populations.py[174]:_set_initial_value_array)
2020-12-02 15:14:11,376 WARNING  [PyNN] NEST does not allow setting an initial value for g_in (/home/laurent/.local/lib/python3.8/site-packages/pyNN/nest/populations.py[174]:_set_initial_value_array)


0 Creating cell populations...
0 Initialising membrane potential to random values...
0 Connecting populations...
0 Setting up recording...0%                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                            

/home/laurent/.local/lib/python3.8/site-packages/pyNN/recording/__init__.py:264: UserWarning:Recorded at least one spike after t_stop



--- Vogels-Abbott Network Simulation ---
Nodes                  : 1
Number of Neurons      : 4000
Number of Synapses     : 128066 e→e,i  32011 i→e,i
Excitatory conductance : 4 nS
Inhibitory conductance : 51 nS
Excitatory rate        : 23.1569 Hz
Inhibitory rate        : 21.765 Hz
Build time             : 13.0561 s
Simulation time        : 10.1949 s
Writing time           : 0.952852 s
